In [1]:
import numpy as np

### Below is the base data type class to store vectors/ matrices

In [3]:
class Tensor(object):

    def __init__(self, data):
        super().__init__()
        self.data = np.array(data)

    def __add__(self, other):
        return Tensor(self.data + other.data)

    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

In [5]:
x = Tensor([1,2,3,4,5])
print(x)

y = x + x
print(y)

[1 2 3 4 5]
[ 2  4  6  8 10]


### So in above if we want to add more functions to a tensor we need to create relavant functions inside of the class.
> eg: add function


### In order to include automatic gradient computation we need to keep track of the functions performed on the Tensor and the graph of computations.

### To do that we need to define few other properties to the above tensor class.

In [10]:
class Tensor (object):
    def __init__(self, data, creators=None, creation_op=None):
        self.data = np.array(data)

        # To keep track of the operation performed to the tensor.
        self.creation_op = creation_op

        # To keep track of graph of computation
        self.creators = creators

        # Gradient of this tensor
        self.grad = None

    # Backpropagation
    def backward(self, grad):
        # Given the input gradient we change the current one to that
        self.grad = grad
        if(self.creation_op == "add"):
            # If operation if addition then linear propagation to the creator nodes
            self.creators[0].backward(grad)
            self.creators[1].backward(grad)
        #print(self, self.grad)

    def __add__(self, other):
        return Tensor(self.data + other.data,
                    creators=[self,other],
                    creation_op="add")

    def __repr__(self):
        return str(self.data.__repr__())

    def __str__(self):
        return str(self.data.__str__())

### Below image shows the computational graph created by addition of 2 Tensors. This is how computational graphs are getting created.

<center><img src="images\computational_graph.png" alt="Computational Graph"></center>

In [12]:
x = Tensor([1,2,3,4,5])
y = Tensor([2,2,2,2,2])

z = x + y
z.backward(Tensor(np.array([1,1,1,1,1])))

print("X", x, x.grad)
print("Y", y, y.grad)
print("Z", z, z.grad)

X [1 2 3 4 5] [1 1 1 1 1]
Y [2 2 2 2 2] [1 1 1 1 1]
Z [3 4 5 6 7] [1 1 1 1 1]


### Above implementation have a bug where if a Tensor get gradient updates from more than 2 paths gradient update would be wrong. (like in the below computation graph). In order to fix that instead of replacing the grad value we update it accordingly.

In [19]:
class Tensor (object):
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):  # ID to uniquely identify the Tensor

        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {} # To keep track of Tensor's child tensors

        # Generate unique integer as an ID to the Tensor
        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id

        # When creating a tensor, if creators are available, 
        #       we modify the creator children dictionary to keep track of newly created child.
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    # Helper function to check whether all gradients have backpropagated from the child Tensors.
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True


    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            # Accumulate gradients from all the paths and add them up
            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad

            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)


    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(  self.data + other.data,
                            autograd=True,
                            creators=[self,other],
                            creation_op="add")
        return Tensor(self.data + other.data)


    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

<center><img src="images\multipath_computational_graph.png" alt="Computational Graph"></center>

### Use the above image to understand the computatioanl graph and how the variable values change thorughout the process.

In [20]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + b
e = b + c
f = d + e

f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([2,2,2,2,2]))

[ True  True  True  True  True]


### If we need to add support to additional mathematical operations all we need to do is that adding the respective function and its derivative to the backpropagation logic. Check below for the implementation of "Negation".

In [30]:
class Tensor (object):
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):  # ID to uniquely identify the Tensor

        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {} # To keep track of Tensor's child tensors

        # Generate unique integer as an ID to the Tensor
        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id

        # When creating a tensor, if creators are available, 
        #       we modify the creator children dictionary to keep track of newly created child.
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    # Helper function to check whether all gradients have backpropagated from the child Tensors.
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True


    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            # Accumulate gradients from all the paths and add them up
            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad

            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)
                if(self.creation_op == "neg"):
                    # Taking the negation of the gradient tensor
                    self.creators[0].backward(self.grad.__neg__())


    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(  self.data + other.data,
                            autograd=True,
                            creators=[self,other],
                            creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(  self.data * -1,
                            autograd=True,
                            creators=[self],
                            creation_op="neg")
        return Tensor(self.data * -1)

    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

In [31]:
a = Tensor([1,2,3,4,5], autograd=True)
b = Tensor([2,2,2,2,2], autograd=True)
c = Tensor([5,4,3,2,1], autograd=True)

d = a + (-b)
e = (-b) + c
f = d + e

# Note we are sending a Tensor as gradient
f.backward(Tensor(np.array([1,1,1,1,1])))
print(b.grad.data == np.array([-2,-2,-2,-2,-2]))

[ True  True  True  True  True]


### Now below include Tensor class implementation with support for few other mathematical functions.
* substraction
* multiplication
* sum (Sums values across a dimention)
* expand (Function that copies data along a dimension)
* transpose
* mm (matrix multiplication)

> Matrix Multiplication related backpropagation implementation is not intuitive. Therefore need to check after writing down the forward propagation steps.

In [145]:
class Tensor (object):
    def __init__(self,data,
                 autograd=False,
                 creators=None,
                 creation_op=None,
                 id=None):  # ID to uniquely identify the Tensor

        self.data = np.array(data)
        self.creators = creators
        self.creation_op = creation_op
        self.grad = None
        self.autograd = autograd
        self.children = {} # To keep track of Tensor's child tensors

        # Generate unique integer as an ID to the Tensor
        if(id is None):
            id = np.random.randint(0,100000)
        self.id = id

        # When creating a tensor, if creators are available, 
        #       we modify the creator children dictionary to keep track of newly created child.
        if(creators is not None):
            for c in creators:
                if(self.id not in c.children):
                    c.children[self.id] = 1
                else:
                    c.children[self.id] += 1

    # Helper function to check whether all gradients have backpropagated from the child Tensors.
    def all_children_grads_accounted_for(self):
        for id, cnt in self.children.items():
            if(cnt != 0):
                return False
        return True


    def backward(self,grad=None, grad_origin=None):
        if(self.autograd):
            if(grad_origin is not None):
                if(self.children[grad_origin.id] == 0):
                    raise Exception("cannot backprop more than once")
                else:
                    self.children[grad_origin.id] -= 1

            # Accumulate gradients from all the paths and add them up
            if(self.grad is None):
                self.grad = grad
            else:
                self.grad += grad

            if(self.creators is not None and (self.all_children_grads_accounted_for() or grad_origin is None)):
                if(self.creation_op == "add"):
                    self.creators[0].backward(self.grad, self)
                    self.creators[1].backward(self.grad, self)

                if(self.creation_op == "neg"):
                    # Taking the negation of the gradient tensor
                    self.creators[0].backward(self.grad.__neg__())

                if(self.creation_op == "sub"):
                    new = Tensor(self.grad.data)
                    self.creators[0].backward(new, self)
                    new = Tensor(self.grad.__neg__().data)
                    self.creators[1].backward(new, self)

                if(self.creation_op == "mul"):
                    new = self.grad * self.creators[1]
                    self.creators[0].backward(new , self)
                    new = self.grad * self.creators[0]
                    self.creators[1].backward(new, self)

                if(self.creation_op == "mm"):
                    act = self.creators[0]
                    weights = self.creators[1]

                    # This isequivalent to --> layer_1_delta=layer_2_delta.dot(weights_1_2.T) part
                    new = self.grad.mm(weights.transpose()) 
                    act.backward(new)
                    new = self.grad.transpose().mm(act).transpose()
                    weights.backward(new)

                if(self.creation_op == "transpose"):
                    self.creators[0].backward(self.grad.transpose())

                if("sum" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    ds = self.creators[0].data.shape[dim]
                    self.creators[0].backward(self.grad.expand(dim,ds))

                if("expand" in self.creation_op):
                    dim = int(self.creation_op.split("_")[1])
                    self.creators[0].backward(self.grad.sum(dim))


    def __add__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(  self.data + other.data,
                            autograd=True,
                            creators=[self,other],
                            creation_op="add")
        return Tensor(self.data + other.data)

    def __neg__(self):
        if(self.autograd):
            return Tensor(  self.data * -1,
                            autograd=True,
                            creators=[self],
                            creation_op="neg")
        return Tensor(self.data * -1)

    def __sub__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(  self.data - other.data,
                            autograd=True,
                            creators=[self,other],
                            creation_op="sub")
        return Tensor(self.data - other.data)

    def __mul__(self, other):
        if(self.autograd and other.autograd):
            return Tensor(  self.data * other.data,
                            autograd=True,
                            creators=[self,other],
                            creation_op="mul")
        return Tensor(self.data * other.data)

    def sum(self, dim):
        if(self.autograd):
            return Tensor(  self.data.sum(dim), # Getting the sum over desired dimension
                            autograd=True,
                            creators=[self],
                            creation_op="sum_" + str(dim))
        return Tensor(self.data.sum(dim))

    def expand(self, dim, copies):
        trans_cmd = list(range(0, len(self.data.shape)))
        trans_cmd.insert(dim, len(self.data.shape))

        new_shape = list(self.data.shape) + [copies]

        new_data = self.data.repeat(copies).reshape(new_shape)
        new_data = new_data.transpose(trans_cmd)

        if(self.autograd):
            return Tensor(  new_data,
                            autograd=True,
                            creators=[self],
                            creation_op="expand_"+str(dim))
        return Tensor(new_data)

    def transpose(self):
        if(self.autograd):
            return Tensor(  self.data.transpose(),
                            autograd=True,
                            creators=[self],
                            creation_op="transpose")
        return Tensor(self.data.transpose())

    def mm(self, x):
        if(self.autograd):
            return Tensor(  self.data.dot(x.data),
                            autograd=True,
                            creators=[self,x],
                            creation_op="mm")
        return Tensor(self.data.dot(x.data))


    def __repr__(self):
        return str(self.data.__repr__())
    def __str__(self):
        return str(self.data.__str__())

### Now we can compare the original vs framework based implementation of a Neural Network

> ### Regular implementation

In [164]:
np.random.seed(0)

data = np.array([[0,0],[0,1],[1,0],[1,1]])
target = np.array([[0],[1],[0],[1]])

weights_0_1 = np.random.rand(2,3)
weights_1_2 = np.random.rand(3,1)

for i in range(10):
    # Forward pass
    layer_1 = data.dot(weights_0_1)
    layer_2 = layer_1.dot(weights_1_2)

    # Loss calculation
    diff = (layer_2 - target)
    sqdiff = (diff * diff)
    loss = sqdiff.sum(0)

    # Backpropagation
    layer_1_grad = diff.dot(weights_1_2.transpose())
    weight_1_2_update = layer_1.transpose().dot(diff)
    weight_0_1_update = data.transpose().dot(layer_1_grad)

    # Weight Updation
    weights_1_2 -= weight_1_2_update * 0.1
    weights_0_1 -= weight_0_1_update * 0.1
    print(loss[0])

5.066439994622396
0.4959907791902341
0.4180671892167177
0.35298133007809646
0.2972549636567376
0.24923260381633278
0.20785392075862477
0.17231260916265181
0.14193744536652994
0.11613979792168387


> ### Framework based implementation

In [165]:
np.random.seed(0)

data = Tensor(np.array([[0,0],[0,1],[1,0],[1,1]]), autograd=True)
target = Tensor(np.array([[0],[1],[0],[1]]), autograd=True)

w = list()
w.append(Tensor(np.random.rand(2,3), autograd=True))
w.append(Tensor(np.random.rand(3,1), autograd=True))

for i in range(10):
    # Forward pass
    pred = data.mm(w[0]).mm(w[1])

    # Calculate Loss
    loss = ((pred - target)*(pred - target)).sum(0)

    # back propagation calculation
    loss.backward(Tensor(np.ones_like(loss.data)))

    # Weight Updation
    for w_ in w:
        w_.data -= w_.grad.data * 0.1
        w_.grad.data *= 0

    print(loss)

[0.58128304]
[0.48988149]
[0.41375111]
[0.34489412]
[0.28210124]
[0.2254484]
[0.17538853]
[0.1324231]
[0.09682769]
[0.06849361]
